In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('avito_wo_nan.csv')
data = data.drop(['Unnamed: 0', 'Руль', 'Цвет', 'Объём двигателя'], axis=1)
probeg = data['Пробег'].values
data['Супер пробег'] =np.log(1 / (probeg + 1))
data = data.drop(['VIN или номер кузова', 'Ссылка'], axis=1)
data = data.loc[data['Тип кузова'] != 'лимузин']
data = data.loc[data['Цена'] > np.quantile(data['Цена'].values, 0.025)]
data = data.loc[data['Цена'] < np.quantile(data['Цена'].values, 0.975)]
data = data.loc[data['Пробег'] < np.quantile(data['Пробег'].values, 0.95)]
data = data.loc[data['Состояние'] != ' битый']




data_dummies = pd.get_dummies(data=data, columns=['Марка', 'Модель', 'Тип кузова',
                                                      'Привод', 'Тип двигателя',
                                                      'Состояние', 'Коробка передач'])

X = data_dummies.drop(['Цена'], axis=1).values
y = data_dummies['Цена'].values

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

transformer = Normalizer().fit(X) # fit does nothing.
transformer.transform(X)

array([[1.25614804e-02, 6.24949276e-06, 2.12482754e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.88006907e-02, 2.43152420e-05, 1.09418589e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.27951192e-02, 1.27377991e-05, 1.34383781e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.19753260e-02, 1.19039026e-05, 9.70168063e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.07128053e-02, 2.05892697e-05, 1.67802548e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.38100609e-02, 2.06120312e-05, 1.58712640e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.2, random_state=42)

In [6]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

clf = Ridge()

alphas = [0.01, 0.1, 1, 10, 100, 1000]
param_grid = {"alpha": alphas}

grid = GridSearchCV(clf, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.028851e-16
  overwrite_a=True).T
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.040493e-16
  overwrite_a=True).T
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.033413e-16
  overwrite_a=True).T
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.040513e-16
  overwrite_a=True).T
/usr/local/lib/python3.7

CPU times: user 31 s, sys: 15 s, total: 46 s
Wall time: 13.9 s


In [7]:
print(grid.best_params_)
print(grid.best_score_)

0.8613616535842178

In [ ]:
%%time 
from sklearn.svm import SVR

clf = SVR ()

kernels = ['poly', 'rbf']
degrees = [3, 5, 7]

param_grid = {"kernel": kernels, "degree": degrees}

grid = GridSearchCV(clf, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

reg_las = Lasso()

alphas = [0.01, 0.1, 1, 10, 100, 1000]
param_grid = {"alpha": alphas}

grid_las = GridSearchCV(reg_las, param_grid=param_grid, cv=5)
grid_las.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [14]:
print(grid_las.best_params_)
print(grid_las.best_score_)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.73490086915673
CPU times: user 15.7 s, sys: 71.6 ms, total: 15.8 s
Wall time: 15.8 s


In [16]:
%%time
from sklearn.model_selection import GridSearchCV

reg = RandomForestRegressor(n_estimators=100)

min_samples_leaf_ = [10, 50, 100, 500, 1000]
max_features_ = ['auto', 'sqrt', 'log2']
min_impurity_decrease_ = [0.001, 0.01, 0.1]

param_grid = {"min_samples_leaf": min_samples_leaf_,\
              "max_features": max_features_,\
              "min_impurity_decrease": min_impurity_decrease_}

grid = GridSearchCV(reg, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)

CPU times: user 1h 55min 18s, sys: 28 s, total: 1h 55min 46s
Wall time: 1h 55min 56s


In [17]:
grid.best_score_

0.8982800888938682

In [18]:
grid.best_params_

{'max_features': 'auto', 'min_impurity_decrease': 0.1, 'min_samples_leaf': 10}